In [5]:
import numpy as np
from numpy.core.defchararray import translate

In [1]:
import src.triangle_ray_intersection as triangle_ray_intersection

In [7]:
names = ['thorax', 'neck', 'a_L1', 'a_L2', 'a_R1', 'a_R2', 'leg_f_L1', 'leg_f_L2', 'leg_f_R1', 'leg_f_R2', 'leg_m_L1', 'leg_m_L2', 'leg_m_R1', 'leg_m_R2', 'a_R0', 'a_L0', 'leg_f_R0', 'leg_f_L0', 'leg_m_R0', 'leg_m_L0', 'eye_L', 'eye_R', 'm_L0', 'm_R0', 'm_L1', 'm_R1']

In [2]:
t = triangle_ray_intersection.CollisionDetector(obj_folder="./example_obj_folder/seed/", skeleton_json_path="", pose_csv="ant_3d_validrows.csv")

In [ ]:
for i in range(2400):
    try:
        location, norm = t.get_collisions(i)
    except:
        location, norm = [], []
    if len(location) > 0:
        print(i)

In [ ]:

scene = t.visualise_collision(1752)

scene.show(viewer="gl")

In [ ]:
scene = t.animal.visualise_animal(1064)
scene.show(viewer="gl", line_settings={'point_size':10})

In [4]:
p = t.animal.get_animal_pose(1064)

[531, 1980]

In [3]:
V = triangle_ray_intersection.Viewer(t.animal, t.obj, 563)

In [30]:
V.scene_widget1.scene.geometry['thorax'].apply_translation([2,2,2])

<trimesh.Trimesh(vertices.shape=(642, 3), faces.shape=(1280, 3))>

In [31]:
V.scene_widget1.do_draw()

In [7]:
v=V.scene_widget1.scene.geometry


In [6]:
V.scene_widget1.scene.delete_geometry(v)